# A class for Gaussian distributed random vectors
A vector with a covariance matrix (error bars along the diagonal).

In [1]:
from numpy        import array, mean, cov, isscalar, asarray
from numpy.linalg import cholesky, inv
from numpy.random import randn

Some relevant infos about how to implement it correctly:

* why to subclass `ndarray`: 
  https://stackoverflow.com/questions/38229953/array-and-rmul-operator-in-python-numpy
* why to implement `__new__`:
  https://numpy.org/doc/stable/user/basics.subclassing.html

Here we keep it a bit hacky...

In [2]:
# helper functions for slicing of two-D arrays
def pick2(A, idx0, idx1=None):
    if idx1 is None:
        idx1 = idx0
    idx0 = [[i] for i in idx0]   # weird numpy slicing
    # https://stackoverflow.com/questions/4257394/slicing-of-a-numpy-2d-array-or-how-do-i-extract-an-mxm-submatrix-from-an-nxn-ar
    return A[idx0, idx1]

def inverse_range(l, idx):
    return [i for i in range(l) if i not in idx]

In [3]:
class GaussianRV():
    
    def __init__(self, mu, Sigma):
        if isscalar(mu):
            mu = asarray([[mu]])
        if isscalar(Sigma):
            Sigma = asarray([[Sigma]])
        self.mu    = mu         # a vector
        self.Sigma = Sigma      # must be a matrix
        self.L     = cholesky(Sigma)   # L == sqrt(Sigma)
        
    def __repr__(self):
        s = f"GaussianRV of shape {self.mu.shape}"
        if self.mu.shape[0] < 7:
            s += f"\nparameter mu    = \n{self.mu}"
            s += f"\nparameter Sigma = \n{self.Sigma}"
        return s
    
    def __add__(self, other):
        # overload self + other
        if isinstance(other, GaussianRV):
            # adding another GaussianRV
            return GaussianRV(self.mu    + other.mu, 
                              self.Sigma + other.Sigma)
        else:
            # adding a constant
            return GaussianRV(self.mu + other, self.Sigma)
    
    def __minus__(self, other):
        # overload self - other
        # note that the variances are added
        return GaussianRV(self.mu    - other.mu,
                          self.Sigma + other.Sigma)
    
    def __mul__(self, other):
        # for scalar multiplication
        # overload self * other
        return GaussianRV(other*self.mu,
                          other*other*self.Sigma)
        
    def __rmul__(self, other):
        # overload other * self
        return self * other
    
    def sample(self, n=1):
        return self.L @ randn(self.mu.shape[0], n) + self.mu

    def lin_trafo(self, A, b=None):
        # linear transformation of a Gaussian
        Y = GaussianRV(A @ self.mu, 
                       A @ self.Sigma @ A.T)
        if b is not None:
            Y = Y + b
        return Y
        
    def sum(self, keep_idx):
        # 'keep_idx' contains the indices to keep,
        # i.e. all others are summed out
                return GaussianRV(mu[keep_idx], pick2(Sigma, keep_idx))
        
    def cond(self, cond_idx, cond_values):
        # let's use the notation of section06/slide21
        assert (cond_values.shape == self.mu[cond_idx].shape), "shape of conditional does not match"
        keep_idx = inverse_range(self.mu.shape[0], cond_idx)
        mu, nu   = self.mu[keep_idx], self.mu[cond_idx]
        print(mu)
        print(nu)
        A, C     = pick2(Sigma, keep_idx), pick2(Sigma, cond_idx)
        B        = pick2(Sigma, keep_idx, cond_idx)
        return GaussianRV(mu + B @ inv(C) @ (cond_value - nu),  # cond mean
                          A  - B @ inv(C) @ B.T)                # cond var


In [4]:
# test
mu    = randn(5,1)
A     = randn(5,5)
Sigma = A@A.T   # symmetrize A
X     = GaussianRV(mu, Sigma)
print(X)

GaussianRV of shape (5, 1)
parameter mu    = 
[[ 0.66267148]
 [ 0.27206714]
 [ 0.8422861 ]
 [ 1.37390099]
 [-0.77992438]]
parameter Sigma = 
[[ 3.80043127  1.80154868 -1.7010485   1.49624942 -0.46830552]
 [ 1.80154868  1.38705336 -1.84554389  0.63643428  0.16455748]
 [-1.7010485  -1.84554389  7.19450202 -0.42300071  2.39059072]
 [ 1.49624942  0.63643428 -0.42300071  1.90598387  0.1332269 ]
 [-0.46830552  0.16455748  2.39059072  0.1332269   3.14668367]]


In [5]:
# sum out some of the coordinates
keep_idx = [0, 2]
Y = X.sum(keep_idx)
Y

GaussianRV of shape (2, 1)
parameter mu    = 
[[0.66267148]
 [0.8422861 ]]
parameter Sigma = 
[[ 3.80043127 -1.7010485 ]
 [-1.7010485   7.19450202]]

In [6]:
# condition on a value
cond_idx = [1,3,4]
cond_value = array([[0.0], [0.0], [0.0]])
Z = X.cond(cond_idx, cond_value)
print(Z)                   

[[0.66267148]
 [0.8422861 ]]
[[ 0.27206714]
 [ 1.37390099]
 [-0.77992438]]
GaussianRV of shape (2, 1)
parameter mu    = 
[[-0.40250683]
 [ 1.58765277]]
parameter Sigma = 
[[1.02320762 1.10626016]
 [1.10626016 2.47382562]]


In [7]:
# sample from the conditional distribution
print(Z)
z = Z.sample(10000)
print('empirical parameters')
print(mean(z, axis=1))   # calling `mean` from numpy
print(cov(z))            # calling `cov` from numpy

GaussianRV of shape (2, 1)
parameter mu    = 
[[-0.40250683]
 [ 1.58765277]]
parameter Sigma = 
[[1.02320762 1.10626016]
 [1.10626016 2.47382562]]
empirical parameters
[-0.37959812  1.58730226]
[[1.01698747 1.08896968]
 [1.08896968 2.43522268]]


In [8]:
# linear transformation
A = randn(3, 5)
b = randn(3, 1)
Z = X.lin_trafo(A, b)
print(Z)

GaussianRV of shape (3, 1)
parameter mu    = 
[[-1.06848857]
 [ 3.71381063]
 [-0.79773508]]
parameter Sigma = 
[[ 35.57624142  30.27095337 -11.05715939]
 [ 30.27095337  34.64040413 -14.1346694 ]
 [-11.05715939 -14.1346694    6.77105343]]
